In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

![](https://i.ytimg.com/vi/Cjv9Xaqnjx4/maxresdefault.jpg)youtube.com

<h1><span class="label label-default" style="background-color:black;border-radius:100px 100px; font-weight: bold; font-family:Garamond; font-size:20px; color:#03e8fc; padding:10px">ROI-Based Processing</span></h1><br>

Define and operate on regions of interest (ROI)

"A region of interest (ROI) is a portion of an image that you want to filter or operate on in some way. You can represent an ROI as a binary mask image. In the mask image, pixels that belong to the ROI are set to 1 and pixels outside the ROI are set to 0. The toolbox offers several options to specify ROIs and create binary masks."

"The toolbox supports a set of objects that you can use to create ROIs of many shapes, such circles, ellipses, polygons, rectangles, and hand-drawn shapes. After you create the objects, you can modify their shape, position, appearance, and behavior." 

https://www.mathworks.com/help/images/roi-based-processing.html

In [ ]:
# Importing essential libraries
import cv2, glob, os
from matplotlib import pyplot as plt
from PIL import Image

In [ ]:
# Importing essential datasets
BASE = '../input/hotel-id-to-combat-human-trafficking-2022-fgvc9/train_images/'

#Reading and understanding a single image

In [ ]:
# Reading and understanding a single image
img = cv2.imread("../input/hotel-id-to-combat-human-trafficking-2022-fgvc9/train_images/100297/000003292.jpg")
plt.figure(figsize=(10,10))
plt.imshow(img, cmap = 'winter'), plt.axis('off'), plt.title('Hotel Room',fontsize=20),plt.show()


In [ ]:
# Loading a whole bunch of images in train and test datasets
image_train = [cv2.imread(file) for file in glob.glob(BASE+'100297/000003292.jpg')]
#image_test = [cv2.imread(file) for file in glob.glob(BASE+'abc.jpg')]
image_01 = cv2.imread(BASE+'10054/000039550.jpg')
image_02 = cv2.imread(BASE+'102048/000003820.jpg')

#Select some images

In [ ]:
# Visualizing the selected images
plt.figure(figsize=(20,10))
plt.subplot(121),plt.imshow(image_01, cmap = 'hsv'), plt.axis('off'), plt.title('Hotel Room 2',fontsize=30)
plt.subplot(122),plt.imshow(image_02, cmap = 'cividis'), plt.axis('off'), plt.title('Hotel room 3',fontsize=30)
plt.show()

#Histogram 1st image

In [ ]:
# Visualizing the image histogram for first image
counts,bins,_ = plt.hist(image_01.ravel(),density = False, alpha = 0.8, histtype = 'stepfilled', color = '#0303FF', edgecolor = '#44FF80')

#Histogram 2nd image

In [ ]:
# Visualizing the histogram for second image
counts,bins,_ = plt.hist(image_02.ravel(),density = True, alpha = 0.2, histtype = 'stepfilled', color = '#DC143C', edgecolor = '#FF0000')

#Multivariate normal - 1st image

In [ ]:
# Understanding multivariate normal for the first image
x, y = np.random.multivariate_normal([0,200],[[1, 0], [0, 200]],10000).T
plt.hist2d(x,y,bins=30,cmap="Blues")
cb = plt.colorbar()
cb.set_label('Counts in Bin')
plt.show()

#Multivariate normal - 2nd image

In [ ]:
# Understanding multivariate normal for the second image
x, y = np.random.multivariate_normal([0,200],[[1, 0], [0, 200]],10000).T
plt.hist2d(x,y,bins=30,cmap="Greens")
cb = plt.colorbar()
cb.set_label('Counts in Bin')
plt.show()

#Grayscale Histogram

In [ ]:
# Grayscale histogram
plt.figure(figsize=(15,8))
plt.subplot(241), plt.plot(cv2.calcHist([cv2.cvtColor(image_01, cv2.COLOR_BGR2GRAY)],[0],None,[256], [0,256]), color = 'k'), plt.title('Hotel Room 2',fontsize=15)
plt.subplot(242), plt.plot(cv2.calcHist([image_01],[0],None,[256],[0,256]),color = 'b'), plt.xlim([0,256])
plt.subplot(243), plt.plot(cv2.calcHist([image_01],[0],None,[256],[0,256]),color = 'g'), plt.xlim([0,256])
plt.subplot(244), plt.plot(cv2.calcHist([image_01],[0],None,[256],[0,256]),color = 'r'), plt.xlim([0,256])
plt.subplot(245), plt.plot(cv2.calcHist([cv2.cvtColor(image_02, cv2.COLOR_BGR2GRAY)],[0],None,[256], [0,256]), color = 'k'), plt.title('Hotel Room 3',fontsize=15)
plt.subplot(246), plt.plot(cv2.calcHist([image_02],[0],None,[256],[0,256]),color = 'b'), plt.xlim([0,256])
plt.subplot(247), plt.plot(cv2.calcHist([image_02],[0],None,[256],[0,256]),color = 'g'), plt.xlim([0,256])
plt.subplot(248), plt.plot(cv2.calcHist([image_02],[0],None,[256],[0,256]),color = 'r'), plt.xlim([0,256])
plt.show()

#Grayscale

In [ ]:
# Grayscale Histogram Equalization
plt.figure(figsize=(20,10))
plt.subplot(121),plt.imshow(cv2.cvtColor(image_01, cv2.COLOR_BGR2GRAY), cmap = 'gray'), plt.axis('off'), plt.title('Hotel room 2',fontsize=20)
plt.subplot(122),plt.imshow(cv2.equalizeHist(cv2.cvtColor(image_01, cv2.COLOR_BGR2GRAY)), cmap = 'gray'), plt.axis('off'), plt.title('Equalized Histogram',fontsize=20)
plt.show()

#Channel Histogram Equalization

In [ ]:
# 3-channel Histogram Equalization
channels = cv2.split(image_01)
eq_channels = []
for ch, color in zip(channels, ['B', 'G', 'R']): 
    eq_channels.append(cv2.equalizeHist(ch))
plt.figure(figsize=(20,10))
plt.subplot(121),plt.imshow(image_01, cmap = 'gray'), plt.axis('off'), plt.title('Hotel room 2',fontsize=20)
plt.subplot(122),plt.imshow(cv2.cvtColor(cv2.merge(eq_channels),cv2.COLOR_BGR2RGB), cmap = 'gray'), plt.axis('off'), plt.title('Equalized Histogram',fontsize=20)
plt.show()

#Blurring

In [ ]:
# Averaging the images
plt.figure(figsize=(20,10))
plt.subplot(121),plt.imshow(cv2.blur(image_01,(40,40)), cmap = 'hsv'), plt.axis('off'), plt.title('Hotel room 2',fontsize=30)
plt.subplot(122),plt.imshow(cv2.blur(image_02,(20,20)), cmap = 'cividis'), plt.axis('off'), plt.title('Hotel room 3',fontsize=30)
plt.show()

#ROI (Region of Interest) selection.

In [ ]:
# ROI selection in image the images
# image_01[300:600,170:400] where the first is from top to bottom and the second is from left to right
lugagge_01 = image_01[380:480,180:390]
lugagge_02 = image_02[380:480,180:390] # Original 420:500,190:310
plt.figure(figsize=(20,10))
plt.subplot(121), plt.imshow(lugagge_01, cmap = 'hsv'), plt.axis('off'), plt.title('Hotel room 2 lugagge',fontsize=30)
plt.subplot(122), plt.imshow(desk_02, cmap = 'Blues'), plt.axis('off'), plt.title('Hotel room 3 lugagge',fontsize=30)
plt.show()

#Image information

In [ ]:
# Randomly getting some image information
print(image_01.shape)
print(image_01.dtype)
print(lugagge_01.shape)
print(lugagge_02.shape)

#Borders

In [ ]:
# Making borders for the images
plt.figure(figsize=(20,10))
plt.subplot(231), plt.imshow(lugagge_01, cmap = 'gray'), plt.axis('off'), plt.title('Grey',fontsize=25)
plt.subplot(232), plt.imshow(cv2.copyMakeBorder(lugagge_01,10,10,10,10,cv2.BORDER_REPLICATE), cmap = 'Blues'), plt.axis('off'), plt.title('Replicate',fontsize=25)
plt.subplot(233), plt.imshow(cv2.copyMakeBorder(lugagge_01,10,10,10,10,cv2.BORDER_REFLECT), cmap = 'gray'), plt.axis('off'), plt.title('Reflect',fontsize=25)
plt.subplot(234), plt.imshow(cv2.copyMakeBorder(lugagge_01,10,10,10,10,cv2.BORDER_REFLECT_101), cmap = 'Blues'), plt.axis('off'), plt.title('Reflect 101',fontsize=25)
plt.subplot(235), plt.imshow(cv2.copyMakeBorder(lugagge_01,10,10,10,10,cv2.BORDER_WRAP), cmap = 'gray'), plt.axis('off'), plt.title('Wrap',fontsize=25)
plt.subplot(236), plt.imshow(cv2.copyMakeBorder(lugagge_01,10,10,10,10,cv2.BORDER_CONSTANT,value=(120,80,250)), cmap = 'Blues'), plt.axis('off'), plt.title('Constant',fontsize=25)
plt.subplots_adjust(wspace=0.05, hspace=-0.3)
plt.show()

#Gaussian Masking

In [ ]:
# Mask operations for the images
kernel = cv2.getGaussianKernel(15, 2.0)
kernel_2D = kernel @ kernel.transpose()
blurred_lugagge = cv2.filter2D(lugagge_01, -1, kernel_2D)
plt.imshow(blurred_lugagge, cmap = 'Blues'), plt.axis('off'), plt.title('Gaussian masking',fontsize=20), plt.show()

#Blending images

In [ ]:
# Blending images
#dst = cv2.addWeighted(src1, alpha, src2, beta, gamma[, dst[, dtype]])
#dst = src1 * alpha + src2 * beta + gamma
lugagge_02x = cv2.resize(lugagge_02,lugagge_01.shape[1::-1])
blended_image = cv2.addWeighted(lugagge_01, 0.5, lugagge_02x, 0.5, 0)
plt.imshow(blended_image, cmap = 'Blues'), plt.axis('off'), plt.title('Merging images using weights',fontsize=20), plt.show()

#Masking Images - Bitwise Masking

In [ ]:
# Masking images
plt.imshow(cv2.bitwise_and(lugagge_01, lugagge_02x), cmap = 'Blues'), plt.axis('off'), plt.title('Bitwise masking',fontsize=20), plt.show()

#Masking Images

In [ ]:
# Masking images
plt.imshow((lugagge_01*0.2+lugagge_02x*0.8).astype(np.uint8), cmap = 'Blues'), plt.axis('off'), plt.title('Masking images',fontsize=20), plt.show()

#Uniform addition of pixel values to images

In [ ]:
# Uniform addition of pixel values to images
lugagge_01x = (lugagge_01 * 0.5 + lugagge_02x * 0.2 + (96, 128, 160)).clip(0,255)
plt.imshow(lugagge_01x.astype(np.uint8), cmap = 'Blues'), plt.axis('off'), plt.title('Uniform addition',fontsize=20), plt.show()

#Sample Mask

In [ ]:
# Mask creation by drawing in image
mask_01 = np.zeros_like(image_02[0:300,0:400])
cv2.rectangle(mask_01, (50, 50), (100, 200), (255, 255, 255), thickness=-1)
cv2.circle(mask_01, (200, 100), 50, (255, 255, 255), thickness=-1)
cv2.fillConvexPoly(mask_01, np.array([[330, 50], [300, 200], [360, 150]]), (255, 255, 255))
mask_01x = cv2.resize(mask_01,image_02.shape[1::-1])
plt.imshow(mask_01), plt.axis('off'), plt.title('Sample Mask',fontsize=20), plt.show()

#Bitwise and Mask created

In [ ]:
# Bitwise and with the mask created
plt.figure(figsize=(10,10))
plt.imshow(cv2.bitwise_and(image_02,mask_01x)), plt.axis('off'), plt.title('Bitwise masking',fontsize=20), plt.show()

In [ ]:
# Reading a new image for working with image channels
image_03 = cv2.imread(BASE+'100297/000003292.jpg')
print(image_03.shape)

# Splitting the channels
plt.figure(figsize=(15,15))
b,g,r = cv2.split(image_03)
mask_03 = np.zeros(image_03.shape[:2], dtype = "uint8")
image_03x = cv2.merge((mask_03,g,r))
plt.subplot(221), plt.imshow(image_03[:,:,0], cmap= 'gray'), plt.axis('off'), plt.title('Red Channel',fontsize=20)
plt.subplot(222), plt.imshow(image_03[:,:,1], cmap= 'gray'), plt.axis('off'), plt.title('Green Channel',fontsize=20) 
plt.subplot(223), plt.imshow(image_03[:,:,2], cmap= 'gray'), plt.axis('off'), plt.title('Blue Channel',fontsize=20) 
plt.subplot(224), plt.imshow(image_03x), plt.axis('off'), plt.title('Channels Merged',fontsize=20)
plt.subplots_adjust(wspace=0, hspace=-0.25)
plt.show()

In [ ]:
# Crop and Resize Images
height, width = image_03.shape[:2]
quarter_height, quarter_width = height / 4, width / 4
T = np.float32([[1, 0, quarter_width], [0, 1, quarter_height]]) 

plt.figure(figsize=(20,15))
plt.subplot(231), plt.imshow(image_03), plt.axis('off'), plt.title('Hotel room 1',fontsize=20)
plt.subplot(232), plt.imshow(cv2.resize(image_03,(200,200))), plt.axis('off'), plt.title('Resized Image',fontsize=20)
plt.subplot(233), plt.imshow(image_03[470:610,420:610]), plt.axis('off'), plt.title('Cropped Image',fontsize=20)
plt.subplot(234), plt.imshow(cv2.warpAffine(image_03, T, (width,height)) ), plt.axis('off'), plt.title('Translated Image',fontsize=20)
plt.subplot(235), plt.imshow(cv2.rotate(image_03, cv2.ROTATE_90_CLOCKWISE)), plt.axis('off'), plt.title('Rotated Image',fontsize=20)
plt.subplot(236), plt.imshow(np.flip(image_03,(0, 1))), plt.axis('off'), plt.title('Flipped Image',fontsize=20);

#Acknowledgement:

Shrushrita Sharma  https://www.kaggle.com/code/shrushritasharma/understanding-basic-image-operations/notebook